In [1]:
import os
import sys
from collections import namedtuple
from operator import attrgetter

from psutil import cpu_count
from gurobipy import *

import pprint

In [2]:
Item = namedtuple('Item', ['index', 'value', 'weight', 'density'])

In [3]:
def solve_it(input_data):
    
    # Modify this code to run your optimization algorithm

    # parse the input
    lines = input_data.split('\n')

    first_line = lines[0].split()
    item_count = int(first_line[0])
    capacity = int(first_line[1])
    
    print('Total Items:', item_count)
    print('Knapsack Capacity:', capacity)

    items = []

    for i in range(1, item_count+1):
        line = lines[i]
        parts = line.split()
        v, w = int(parts[0]), int(parts[1])
        items.append(Item(i-1, v, w, 1.0 * v / w))
        
    print('List Items:')
    pprint.pprint(items)
    
    # greedy solution
    # put items with higher value "density" first
    obj, taken = greedy(capacity, items)
    # prepare the solution in the specified output format
    print("\nSOLUTION (Greedy):")
    output_data = 'Total value: ' + str(obj) + '\n'
    output_data += 'Items taken: '
    output_data += ' '.join(map(str, taken))
    print(output_data)
    
    # dynamic programming solution
    # optimal but high memory utilization with larger problem
    obj, taken = dp(capacity, items)
    
    # prepare the solution in the specified output format
    print("\nSOLUTION (DP):")
    output_data = 'Total value: ' + str(obj) + '\n'
    output_data += 'Items taken: '
    output_data += ' '.join(map(str, taken))
    print(output_data)
    
    # return output_data

In [4]:
def mip():
    pass

def dp(capacity, items):
    
    n = len(items)
    taken = [0] * n
    table_of_values = [[0 for j in range(capacity + 1)] for i in range(n + 1)]
    
    for i in range(n + 1):
        
        if i > 0:
            value = items[i - 1].value
            weight = items[i - 1].weight
            
        for j in range(capacity + 1):
            if i == 0 or j == 0:
                continue
            elif weight > j:
                table_of_values[i][j] = table_of_values[i - 1][j]
            else:
                v_take = table_of_values[i - 1][j - weight] + value
                v_keep = table_of_values[i - 1][j]
                table_of_values[i][j] = max(v_take, v_keep)
    
    total_weight = capacity
    
    for i in reversed(range(n)):
        if table_of_values[i][total_weight] == table_of_values[i + 1][total_weight]:
            continue
        else:
            taken[i] = 1
            total_weight -= items[i].weight

    return table_of_values[-1][-1], taken

def greedy(capacity, items):
    
    n = len(items)
    taken = [0] * n
    filled = 0
    value = 0
    
    for item in sorted(items, key=attrgetter('density')):
        
        filled += item.weight
        
        if filled <= capacity:
            taken[item.index] = 1
            value += item.value
            
    return value, taken

## Solve It!

In [5]:
data_dir = './data/'
data_file = os.listdir(data_dir)[3] # change index to change the data

file_location = data_dir + data_file
with open(file_location, 'r') as input_data_file:
    input_data = input_data_file.read()
    output_data = solve_it(input_data)
    
    # print(output_data)

Total Items: 30
Knapsack Capacity: 100000
List Items:
[Item(index=0, value=90000, weight=90001, density=0.9999888890123443),
 Item(index=1, value=89750, weight=89751, density=0.9999888580628629),
 Item(index=2, value=10001, weight=10002, density=0.9999000199960008),
 Item(index=3, value=89500, weight=89501, density=0.9999888269404811),
 Item(index=4, value=10252, weight=10254, density=0.9998049541642285),
 Item(index=5, value=89250, weight=89251, density=0.9999887956437463),
 Item(index=6, value=10503, weight=10506, density=0.9997144488863506),
 Item(index=7, value=89000, weight=89001, density=0.9999887641711891),
 Item(index=8, value=10754, weight=10758, density=0.9996281836772635),
 Item(index=9, value=88750, weight=88751, density=0.9999887325213237),
 Item(index=10, value=11005, weight=11010, density=0.9995458673932789),
 Item(index=11, value=88500, weight=88501, density=0.9999887006926476),
 Item(index=12, value=11256, weight=11262, density=0.9994672349493873),
 Item(index=13, valu

---